# 将上海市旅游数据月分为日

## 1.数据准备
原则：将之前预测的2019全年数据分配作为上海市旅游入境疫情前全年数据日分配，2020-2024依照异常数据处理原则每年一分配

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import torch

# 创建数据库引擎
database_url = "sqlite:///../data/data.sqlite"
engine = create_engine(database_url)

imputed_data_query = f"SELECT * FROM hk_customs_daily_imputed"
df = pd.read_sql_query(imputed_data_query, engine, index_col='date')
df.index = pd.to_datetime(df.index)

In [2]:
# 计算每个月的总和
monthly_sums = df.resample('ME').sum()
# 初始化一个新的DataFrame来存储每日比例数据
daily_ratios = pd.DataFrame(index=df.index)
# 对于df中的每一列，计算每日比例
for column in df.columns:
    # 对于每个月，计算该月每一天的值占该月总和的比例
    daily_ratios[column] = df[column] / df[column].resample('ME').transform('sum')

In [3]:
regular_slice_start = '2019-01-01'
regular_slice_end = '2019-12-31'
special_slice_start = '2020-01-01'
special_slice_end = '2024-01-31'

regular_ratios = daily_ratios[regular_slice_start:regular_slice_end]
special_ratios = daily_ratios[special_slice_start:special_slice_end]